In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import datasets as misc

from utils import batch_plot
from conv import load_sample_filters, generate_output_size

In [ ]:
sample_image = misc.ascent()
sample_image = sample_image / sample_image.max()
batch_plot(np.expand_dims(sample_image, 0), with_border=False, cmap=plt.cm.gray, imgsize=6)

# Generate sliding window views of the image

<img src="./images/sliding_steps.gif">

In [ ]:
# example of using numpy sliding_window_view: stride=1
height, width, window, stride = 5, 5, 3, 1
x = np.arange(height * width).reshape((height, width))
y = np.lib.stride_tricks.sliding_window_view(x, window_shape=(window, window))

# (height - window)/stride + 1 = chunk_height
chunk_height = generate_output_size(height, window, stride, padding=0)
chunk_width = generate_output_size(width, window, stride, padding=0)
assert y.shape == (chunk_height, chunk_width, window, window)

# low level operation
stride_height, stride_width = x.strides
z = np.lib.stride_tricks.as_strided(
    x,
    shape=(chunk_height, chunk_width, window, window),
    strides=(
        stride * stride_height,
        stride * stride_width,
        stride_height,
        stride_width,
    ),
)
assert np.allclose(y, z)

In [ ]:
# plot nxn views of the image
n = 4
height, width = sample_image.shape
num_stride_height, num_stride_width = height // n, width // n
chunk_height, chunk_width = n, n
stride_height, stride_width = sample_image.strides
# (height - filter_height)/stride + 1 = chunk_height
filter_height = height - num_stride_height * (chunk_height - 1)
filter_width = width - num_stride_width * (chunk_width - 1)
chunks = np.lib.stride_tricks.as_strided(
    sample_image,
    shape=(chunk_height, chunk_width, filter_height, filter_width),
    strides=(
        num_stride_height * stride_height,
        num_stride_width * stride_width,
        stride_height,
        stride_width,
    ),
)
assert chunks.shape == (chunk_height, chunk_width, filter_height, filter_width)

In [ ]:
sliding_sample_image = chunks.reshape((-1, filter_height, filter_width))
batch_plot(
    sliding_sample_image,
    with_border=False,
    cmap=plt.cm.gray,
    tight_layout=None,
    wspace=0.01,
    hspace=0.01,
    imgsize=2,
)

# Apply filters to the sliding window chunks

<img src="./images/convolution_steps.gif">

In [ ]:
# fmt: off
zero = np.array([
    [0 ,0 ,5, 13,9, 1 ,0 ,0 ], 
    [0 ,0 ,13,15,10,15,5 ,0 ], 
    [0 ,3 ,15,2 ,0, 11,8 ,0 ],
    [0 ,4 ,12,0 ,0, 8 ,8 ,0 ],
    [0 ,5 ,8, 0 ,0, 9 ,8 ,0 ],
    [0 ,4 ,11,0 ,1, 12,7 ,0 ],
    [0 ,2 ,14,5 ,10,12,0 ,0 ],
    [0 ,0 ,6, 13,10,0 ,0 ,0 ]
])
# fmt: on

batch_plot(np.expand_dims(zero, 0), cmap=plt.cm.gray_r)

Define gray scale image as $A$, kernel as $W$ and feature map as $Z$,

$$
Z = A*W = \sum_i\sum_j\sum_k\sum_l A(i+k, j+l)W(k,l) \tag {1}
$$

Where,

$$
Z(i,j) = (A*W)(i,j) = \sum_k\sum_l A(i+k, j+l)W(k,l) \tag {2}
$$



In [ ]:
filter_size = 5
zero_height, zero_width = zero.shape
feature_map_height = generate_output_size(zero_height, filter_size, stride=1, padding=0)
feature_map_width = generate_output_size(zero_width, filter_size, stride=1, padding=0)
gaussain_filter = load_sample_filters(size=filter_size, sigma=1)["gaussian"]


"""step by step 
feature_map = np.zeros((feature_map_height, feature_map_width))

for m_h in range(feature_map_height):
    for m_w in range(feature_map_width):
        for f_h in range(filter_size):
            for f_w in range(filter_size):
                feature_map[m_h, m_w] += zero[m_h+f_h, m_w+f_w] * gaussain_filter[f_h, f_w]
"""

# with stride tricks
sliding_view_of_zero = np.lib.stride_tricks.sliding_window_view(zero, window_shape=(filter_size, filter_size))

assert gaussain_filter.shape == (filter_size, filter_size)
assert sliding_view_of_zero.shape == (
    feature_map_height,
    feature_map_width,
    filter_size,
    filter_size,
)

feature_map = (sliding_view_of_zero * gaussain_filter).sum(axis=(2, 3))
batch_plot(np.expand_dims(feature_map, 0), cmap=plt.cm.gray_r)

In [ ]:
filter_height = filter_width = 7
sample_filters = load_sample_filters(size=filter_height, sigma=1)

batch_plot(
    list(sample_filters.values()),
    list(sample_filters.keys()),
    with_border=True,
    cmap=plt.cm.gray_r,
    tight_layout=None,
    wspace=0.1,
    hspace=0.1,
    imgsize=4,
)

In [ ]:
# chunked with stride of 1
height, width = sample_image.shape
filters = np.asarray(list(sample_filters.values()))
chunks = np.lib.stride_tricks.sliding_window_view(sample_image, window_shape=(filter_height, filter_width))
chunk_height, chunk_width = height - filter_height + 1, width - filter_width + 1

assert chunks.shape == (chunk_height, chunk_width, filter_height, filter_width)

# chunks:                                                   (chunk_height, chunk_width, filter_height, filter_width)
# filters:                                                  (num_filters, filter_height, filter_width)

# 1. step by step
# np.expand_dims(chunks, 2):                                (chunk_height, chunk_width, 1, filter_height, filter_width)
# np.expand_dims(chunks, 2) * filters:                      (chunk_height, chunk_width, num_filters, filter_height, filter_width)
# np.expand_dims(chunks, 2) * filters).sum(axis=(-2,-1)):   (chunk_height, chunk_width, num_filters)
# filtered_sample_image = (np.expand_dims(chunks, 2) * filters).sum(axis=(-2,-1)).transpose((2,0,1))
# 2. tensordot
# filtered_sample_image = np.tensordot(chunks, filters, axes=((2,3), (1,2))).transpose((2,0,1))
# 3. einsum
# filtered_sample_image = np.einsum('ijkl,nkl->nij',chunks,filters)

# 4. img2col
# filters.reshape((-1, filter_height*filter_width)).T                           (filter_height*filter_width, num_filters)
# chunks.reshape((chunk_height*chunk_width, filter_height*filter_width))        (chunk_height*chunk_width, filter_height*filter_width)
filtered_sample_image = (
    chunks.reshape((chunk_height * chunk_width, filter_height * filter_width))
    @ filters.reshape((-1, filter_height * filter_width)).T
)
filtered_sample_image = filtered_sample_image.reshape((chunk_height, chunk_width, -1)).transpose((2, 0, 1))

assert filtered_sample_image.shape == (len(filters), chunk_height, chunk_width)

batch_plot(
    filtered_sample_image,
    list(sample_filters.keys()),
    with_border=False,
    cmap=plt.cm.gray,
    tight_layout=None,
    wspace=0.1,
    hspace=0.1,
    imgsize=6,
)

In [ ]:
from scipy import signal


def conv2d(image, kernel):
    kernel_height, kernel_width = kernel.shape
    padding_height, padding_width = (kernel_height - 1) // 2, (kernel_width - 1) // 2
    # same padding
    image = np.pad(image, pad_width=((padding_height, padding_width), (padding_height, padding_width)))
    chunks = np.lib.stride_tricks.sliding_window_view(image, window_shape=kernel.shape)
    return np.einsum("hwij,ij->hw", chunks, kernel)


def fft_conv2d(image, kernel):
    kernel_height, kernel_width = kernel.shape
    padding_height, padding_width = (kernel_height - 1) // 2, (kernel_width - 1) // 2

    output_shape = np.array(image.shape) + np.array(kernel.shape) - 1
    fft_shape = 2 ** np.ceil(np.log2(output_shape)).astype(int)
    return np.real(np.fft.ifft2(np.fft.fft2(image, fft_shape) * np.fft.fft2(kernel, fft_shape)))[
        : output_shape[0], : output_shape[1]
    ][padding_height:-padding_height, padding_width:-padding_width]

In [ ]:
# symmetric
assert np.allclose(
    conv2d(sample_image, sample_filters["gaussian"]),
    signal.fftconvolve(sample_image, sample_filters["gaussian"], mode="same"),
)

assert np.allclose(
    conv2d(sample_image, sample_filters["gaussian"]), fft_conv2d(sample_image, sample_filters["gaussian"])
)

# not symmetric
assert not np.allclose(
    conv2d(sample_image, sample_filters["vertical"]),
    signal.fftconvolve(sample_image, sample_filters["vertical"], mode="same"),
)

assert np.allclose(
    conv2d(sample_image, np.rot90(sample_filters["vertical"], k=2)),
    signal.fftconvolve(sample_image, sample_filters["vertical"], mode="same"),
)

assert np.allclose(
    conv2d(sample_image, np.rot90(sample_filters["vertical"], k=2)),
    fft_conv2d(sample_image, sample_filters["vertical"]),
)

In [ ]:
%timeit conv2d(sample_image, sample_filters['vertical'])
%timeit signal.fftconvolve(sample_image, np.rot90(sample_filters['vertical'], k=2), mode='same')
%timeit fft_conv2d(sample_image, np.rot90(sample_filters['vertical'], k=2))

- [Array programming with NumPy](https://www.nature.com/articles/s41586-020-2649-2)
- [Advanced NumPy - Scipy Lecture Notes](http://scipy-lectures.org/advanced/advanced_numpy/)
- [An Illustrated Guide to Shape and Strides](https://ajcr.net/stride-guide-part-1/)
- [Advanced NumPy: Master stride tricks with 25 illustrated exercises](https://towardsdatascience.com/advanced-numpy-master-stride-tricks-with-25-illustrated-exercises-923a9393ab20)
- [Advanced NumPy | SciPy Japan 2019 Tutorial | Juan Nunuz-Iglesias](https://www.youtube.com/watch?v=cYugp9IN1-Q)
- [Introduction to Numerical Computing with NumPy | SciPy 2019 Tutorial | Alex Chabot-Leclerc](https://www.youtube.com/watch?v=ZB7BZMhfPgk)